Code to warm up kernel

In [ ]:
#Imports
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from pytrends.request import TrendReq
from arch import arch_model
#Declare constants
ETH_TICKER = "ETH-USD"
SPX_TICKER = "^GSPC" 
BTC_TICKER = "BTC-USD"

#Timeframe for data collection
START_DATE = "2022-01-01"
END_DATE = "2025-12-07"


Below is the code to get data for:

Bitcoin & Ether

In [ ]:

def clean_index(df):
    """Safely normalizes index and removes timezone if it exists."""
    df.index = df.index.normalize()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

# --- FETCH ETH DATA ---
# ETH trades 24/7, so this will be your primary (left) dataset
eth_data = yf.Ticker(ETH_TICKER).history(start=START_DATE, end=END_DATE)
df_eth = eth_data[['Open', 'High', 'Low', 'Close','Volume']].copy()
df_combined_daily_eth = clean_index(df_eth)
        
# Display final data types and non-null counts
print("\n--- Final DataFrame Info ---")
df_combined_daily_eth.info()

In [ ]:

# --- FETCH BTC DATA ---
# BTC trades 24/7, so this will be your primary (left) dataset
btc_data = yf.Ticker(BTC_TICKER).history(start=START_DATE, end = END_DATE)
df_btc = btc_data[['Open', 'High', 'Low', 'Close','Volume']].copy()
df_combined_daily_btc = clean_index(df_btc)  

df_combined_daily_btc = df_btc

# Display final data types and non-null counts
print("\n--- Final DataFrame Info ---")
df_combined_daily_btc.info()

BTC HASHRATE + Fear & Greed index ~

All using dataset

In [ ]:
FILE_PATH = r"C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\hashrateBTC.csv"
START_DATE = '2022-01-01 00:00:00'
END_DATE = '2025-12-07 00:00:00'

df_hashrate_data = pd.read_csv(
    FILE_PATH,
    sep=';',
    names = ['DateTime', 'Hash Rate'],
    index_col='DateTime',
    parse_dates=['DateTime'],
    skiprows=1, 
    decimal=',',
    header=None
    )
df_hashrate = df_hashrate_data.loc[START_DATE:END_DATE].copy()


df_hashrate.index = pd.to_datetime(df_hashrate.index)

df_hashrate['Hash Rate'] = pd.to_numeric(df_hashrate['Hash Rate'].astype(float))
daily_avg_hash = df_hashrate.resample('D').mean()
print("successfully extracted hashrate data")

daily_avg_hash.head()


In [ ]:
FILE_PATH_FG = r'C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\btc_with_fgi_4h.csv'##
#START_DATE = '2023-01-01 00:00:00'
#END_DATE = '2025-12-07 00:00:00'
df_fng_data = pd.read_csv(
    FILE_PATH_FG,
    sep=',',
    index_col='timestamp',
    )
df_fng_data.index = pd.to_datetime(df_fng_data.index)                                   
df_fng_data.head()


In [ ]:
df_fng_data.drop(['open','close','high','low','Fear & Greed Classification'], axis=1, inplace=True)
df_fng_data.fillna(method='ffill', inplace=True)

Google Trends data!

In [ ]:
df_gtrends_daily = pd.read_csv(r'C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\google_trends_daily_20251213.csv')
df_gtrends_daily['date'] = pd.to_datetime(df_gtrends_daily['date'])
df_gtrends_daily.set_index('date', inplace=True)

df_gtrends_daily.tail()

In [ ]:
df_gtrends_eth = df_gtrends_daily['GTrends_ETH'].copy()
df_gtrends_eth.head()

In [ ]:
df_gtrends_btc = df_gtrends_daily['GTrends_BTC'].copy()

df_gtrends_btc.info()

Sychronizing and Scaling

In [ ]:
#BTC
df_combined_daily_btc = df_combined_daily_btc.join(df_fng_data)
df_combined_daily_btc = df_combined_daily_btc.join(daily_avg_hash)
df_combined_daily_btc = df_combined_daily_btc.join(df_gtrends_btc)
#ETH
df_combined_daily_eth = df_combined_daily_eth.join(df_fng_data)
df_combined_daily_eth = df_combined_daily_eth.join(df_gtrends_eth)

In [ ]:
df_combined_daily_btc['GTrends_BTC'] = (
    df_combined_daily_btc['GTrends_BTC'].ffill()
)


In [ ]:
df_combined_daily_eth['GTrends_ETH'] = (
    df_combined_daily_eth['GTrends_ETH'].ffill()
)

In [ ]:

# Convert raw HashRate to Exahashes (EH/s) by dividing by 10^18 and volume into billions
df_combined_daily_btc['Volume'] = df_combined_daily_btc['Volume'] / 1e10
df_combined_daily_eth['Volume'] = df_combined_daily_eth['Volume'] / 1e10
df_combined_daily_btc['Hash Rate'] = df_combined_daily_btc['Hash Rate'] / 1e18

# Rename the column so you don't forget the unit later
df_combined_daily_btc.rename(columns={'Hash Rate': 'Hashrate_EH'}, inplace=True)
df_combined_daily_btc.rename(columns={'Volume': 'Volume (billions)'}, inplace=True)
df_combined_daily_eth.rename(columns={'Volume': 'Volume (billions)'}, inplace=True)
print(df_combined_daily_btc.head().to_markdown())
print(df_combined_daily_eth.head().to_markdown())


In [ ]:
print("For ETH:")
df_combined_daily_eth.info()
print("For BTC:")
df_combined_daily_btc.info()

In [ ]:
df_combined_daily_eth['Range'] = (
    df_combined_daily_eth['High'] - df_combined_daily_eth['Low']
)

df_combined_daily_eth['Return'] = (
    df_combined_daily_eth['Close'].pct_change()
)

df_combined_daily_btc['Range'] = (
    df_combined_daily_btc['High'] - df_combined_daily_btc['Low']
)

df_combined_daily_btc['Return'] = (
    df_combined_daily_btc['Close'].pct_change()
)


In [ ]:
df_combined_daily_btc.describe()

In [ ]:
print(df_combined_daily_btc.head().to_markdown())
print(df_combined_daily_eth.head().to_markdown())

In [ ]:

df_combined_daily_eth = df_combined_daily_eth.dropna()
df_combined_daily_btc = df_combined_daily_btc.dropna()
print("ETH shape:", df_combined_daily_eth.shape)
print("BTC shape:", df_combined_daily_btc.shape)

df_combined_daily_eth.head()

In [ ]:
df_eth = df_combined_daily_eth.copy()

df_eth['Direction'] = (
    df_eth['Close'].shift(-1) > df_eth['Close']
).astype(int)

df_eth.dropna(inplace=True)


In [ ]:
df_btc = df_combined_daily_btc.copy()

df_btc['Direction'] = (
    df_btc['Close'].shift(-1) > df_btc['Close']
).astype(int)

df_btc.dropna(inplace=True)


In [ ]:
print("ETH shape:", df_combined_daily_eth.shape)
print("BTC shape:", df_combined_daily_btc.shape)

In [ ]:
df_eth_lag = df_combined_daily_eth.copy()

df_eth_lag['Fear & Greed Index'] = df_eth_lag['Fear & Greed Index'].shift(2)
df_eth_lag['GTrends_BTC'] = df_btc['GTrends_BTC'].shift(2)
df_eth_lag['Momentum'] = df_eth_lag['Close'].shift(1) / df_eth_lag['Close'].shift(5)
df_eth_lag['BTC_Return'] = df_btc['return'].shift(2)

df_eth_lag.dropna(inplace=True)

START OF LEARNING MODEL

Predicting Up vs Stagnant/Down of BTC

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
from sklearn.preprocessing import StandardScaler

BTC

In [ ]:
X = df_eth[
    ['Gtrends_BTC',
     'BTC_Return',
     'Fear & Greed Index',            
     ]]


y = df_eth['Direction'] 



In [ ]:
split_idx = int(len(df_btc) * 0.7)

X_train = X.iloc[:split_idx]
X_test  = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test  = y.iloc[split_idx:]

In [ ]:
print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

print("Train date range:", X_train.index.min(), "→", X_train.index.max())
print("Test date range:", X_test.index.min(), "→", X_test.index.max())

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Model has poor accuracy; attempt to optimise using scaling

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model_scaled = LogisticRegression(max_iter=1000,class_weight='balanced')
model_scaled.fit(X_train_scaled, y_train)

In [ ]:
y_pred_scaled = model_scaled.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_scaled))
print(confusion_matrix(y_test, y_pred_scaled))
print(classification_report(y_test, y_pred_scaled))